In [1]:
# ============================
# SECTION 0: INSTALL & IMPORTS
# ============================

# Install required packages (run once per Colab session)
!pip install interpret shap xgboost -q

# Core libraries
import numpy as np
import pandas as pd

# Models & tools
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    brier_score_loss
)

from interpret.glassbox import ExplainableBoostingClassifier
from xgboost import XGBClassifier
import shap

from scipy.stats import spearmanr


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 38.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.1/780.1 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 17.1 MB/s eta 0:00:00


In [2]:
# =====================================
# SECTION 1: LOAD DATA & BASIC CLEANING
# =====================================

# Path to your cardio dataset (uploaded in Colab)
data_path = "/content/cardio_train.csv"  # change if your file name is different

# Load CSV; let pandas infer delimiter
df = pd.read_csv(data_path, sep=None, engine="python")

print("=== Raw DataFrame Info ===")
df.info()
display(df.head())

# Target column
target_col = "cardio"
if target_col not in df.columns:
    raise ValueError(f"Target column '{target_col}' not found. Columns: {df.columns.tolist()}")

# Drop ID-like columns if present
id_like_cols = [c for c in df.columns if c.lower() in ["id", "index"]]
if id_like_cols:
    print("Dropping ID-like columns:", id_like_cols)
    df = df.drop(columns=id_like_cols)

# Convert obvious binary flags to integer 0/1
binary_cols = ["smoke", "alco", "active", "cardio"]
for col in binary_cols:
    if col in df.columns:
        df[col] = df[col].astype(float).round().astype(int)

# If age is in days (common in cardio datasets), convert to years
if "age" in df.columns and df["age"].median() > 200:
    df["age_years"] = df["age"] / 365.25
    df = df.drop(columns=["age"])
    print("Converted 'age' (days) -> 'age_years' (years).")

# Create BMI if height & weight present
if "height" in df.columns and "weight" in df.columns:
    df["height_m"] = df["height"] / 100.0
    df["BMI"] = df["weight"] / (df["height_m"] ** 2)
    df = df.drop(columns=["height_m"])
    print("Created 'BMI' from height & weight.")

print("\n=== After basic feature engineering ===")
df.info()
display(df.head())

# Define X and y
y = df[target_col]
X = df.drop(columns=[target_col])

feature_names = X.columns.tolist()
print("\nFeature columns:", feature_names)
print("X shape:", X.shape, "  y shape:", y.shape)

# Check class balance of target
print("\n=== Target distribution (cardio) ===")
print(y.value_counts(normalize=True))


=== Raw DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 6.9 MB


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


Dropping ID-like columns: ['id']
Converted 'age' (days) -> 'age_years' (years).
Created 'BMI' from height & weight.

=== After basic feature engineering ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   gender       70000 non-null  int64  
 1   height       70000 non-null  int64  
 2   weight       70000 non-null  float64
 3   ap_hi        70000 non-null  int64  
 4   ap_lo        70000 non-null  int64  
 5   cholesterol  70000 non-null  int64  
 6   gluc         70000 non-null  int64  
 7   smoke        70000 non-null  int64  
 8   alco         70000 non-null  int64  
 9   active       70000 non-null  int64  
 10  cardio       70000 non-null  int64  
 11  age_years    70000 non-null  float64
 12  BMI          70000 non-null  float64
dtypes: float64(3), int64(10)
memory usage: 6.9 MB


,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,age_years,BMI
0,2,168,62.0,110,80,1,1,0,0,1,0,50.357290,21.967120
1,1,156,85.0,140,90,3,1,0,0,1,1,55.381246,34.927679
2,1,165,64.0,130,70,3,1,0,0,0,1,51.627652,23.507805
3,2,169,82.0,150,100,1,1,0,0,1,1,48.249144,28.710479
4,1,156,56.0,100,60,1,1,0,0,0,0,47.841205,23.011177



Feature columns: ['gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'age_years', 'BMI']
X shape: (70000, 12)   y shape: (70000,)

=== Target distribution (cardio) ===
cardio
0    0.5003
1    0.4997
Name: proportion, dtype: float64


In [3]:
# ===========================================
# SECTION 2: TRAIN/TEST SPLIT & IMPUTATION
# ===========================================

# Single fixed split; seeds will vary only for model init, not data split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,      # keeps 50/50 balance in both sets
    random_state=42  # fixed for reproducibility
)

print("Train shape:", X_train.shape, "  Test shape:", X_test.shape)

# Use median imputation for all features (robust, simple)
imputer = SimpleImputer(strategy="median")
X_train_imp = imputer.fit_transform(X_train)
X_test_imp = imputer.transform(X_test)

# Wrap back as DataFrames with column names
X_train_imp_df = pd.DataFrame(X_train_imp, columns=feature_names)
X_test_imp_df = pd.DataFrame(X_test_imp, columns=feature_names)

print("\n=== Any missing values after imputation? ===")
print(pd.DataFrame({
    "train_missing": X_train_imp_df.isna().sum(),
    "test_missing": X_test_imp_df.isna().sum()
}).query("train_missing > 0 or test_missing > 0"))


Train shape: (56000, 12)   Test shape: (14000, 12)

=== Any missing values after imputation? ===
Empty DataFrame
Columns: [train_missing, test_missing]
Index: []


In [4]:
# ===========================================
# SECTION 3: METRICS HELPER (OPTIONAL USE)
# ===========================================

def compute_all_metrics(y_true, y_pred, y_proba):
    """
    Compute core metrics:
      - accuracy
      - roc_auc
      - f1
      - precision
      - recall/sensitivity
      - specificity
      - Brier score
    """
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc = (tp + tn) / (tp + tn + fp + fn)
    roc_auc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall_ = recall_score(y_true, y_pred)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else np.nan
    brier = brier_score_loss(y_true, y_proba)

    return {
        "accuracy": acc,
        "roc_auc": roc_auc,
        "f1": f1,
        "precision": precision,
        "recall_sensitivity": recall_,
        "specificity": specificity,
        "brier": brier
    }


In [7]:
# ============================================================
# SECTION 4: ONE-RUN EXPERIMENT (EBM + XGBOOST + SHAP IMPORTANCE)
# ============================================================

def run_once_and_get_importances(seed,
                                 X_train_df,
                                 X_test_df,
                                 y_train,
                                 y_test,
                                 feature_names):
    """
    Train EBM & XGBoost with a specific random seed.
    Returns:
      - metrics dict
      - EBM global importances (np.array, len = n_features)
      - XGBoost+SHAP global importances (np.array, len = n_features)
    """
    # Numpy arrays for model training
    Xtr = X_train_df.values
    Xte = X_test_df.values

    # -----------------------
    # 1) EBM (interpretable)
    # -----------------------
    ebm = ExplainableBoostingClassifier(random_state=seed)
    ebm.fit(Xtr, y_train)

    ebm_proba = ebm.predict_proba(Xte)[:, 1]
    ebm_pred = (ebm_proba >= 0.5).astype(int)

    ebm_metrics = compute_all_metrics(y_test, ebm_pred, ebm_proba)
    ebm_auc = ebm_metrics["roc_auc"]
    ebm_acc = ebm_metrics["accuracy"]
    ebm_f1 = ebm_metrics["f1"]
    ebm_brier = ebm_metrics["brier"]

    # Global feature importance from EBM
    # EBM does not have a 'feature_importances_' attribute.
    # Instead, we can derive importance from the magnitude of its term scores.
    ebm_importances = np.array([np.sum(np.abs(ebm.term_scores_[i])) for i in range(len(feature_names))])

    # -----------------------
    # 2) XGBoost (black-box)
    # -----------------------
    xgb = XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=seed,
        eval_metric="logloss",
        n_jobs=-1
    )
    xgb.fit(Xtr, y_train)

    xgb_proba = xgb.predict_proba(Xte)[:, 1]
    xgb_pred = (xgb_proba >= 0.5).astype(int)

    xgb_metrics = compute_all_metrics(y_test, xgb_pred, xgb_proba)
    xgb_auc = xgb_metrics["roc_auc"]
    xgb_acc = xgb_metrics["accuracy"]
    xgb_f1 = xgb_metrics["f1"]
    xgb_brier = xgb_metrics["brier"]

    # ------------------------------
    # 3) SHAP global importance
    # ------------------------------
    explainer = shap.TreeExplainer(xgb)
    shap_values = explainer.shap_values(Xtr)

    # Some versions of SHAP return list for multi-class;
    # handle both binary (array) and multi-class (list) cases.
    if isinstance(shap_values, list):
        # Use positive class explanation if it's a list
        shap_values = shap_values[1]

    # Global importance: mean absolute SHAP across samples
    xgb_importances = np.mean(np.abs(shap_values), axis=0)

    # Collect metrics together
    metrics = {
        "seed": seed,
        "EBM_AUC": ebm_auc,
        "EBM_ACC": ebm_acc,
        "EBM_F1": ebm_f1,
        "EBM_Brier": ebm_brier,
        "XGB_AUC": xgb_auc,
        "XGB_ACC": xgb_acc,
        "XGB_F1": xgb_f1,
        "XGB_Brier": xgb_brier
    }

    return metrics, ebm_importances, xgb_importances


In [8]:
# =====================================================
# SECTION 5: MULTI-SEED RUNS (PERFORMANCE + IMPORTANCE)
# =====================================================

N_SEEDS = 20
seeds = list(range(1, N_SEEDS + 1))

metrics_list = []
ebm_importance_runs = []
xgb_importance_runs = []

for s in seeds:
    print(f"Running seed {s} ...")
    metrics, ebm_imp, xgb_imp = run_once_and_get_importances(
        seed=s,
        X_train_df=X_train_imp_df,
        X_test_df=X_test_imp_df,
        y_train=y_train,
        y_test=y_test,
        feature_names=feature_names
    )
    metrics_list.append(metrics)
    ebm_importance_runs.append(ebm_imp)
    xgb_importance_runs.append(xgb_imp)

metrics_df = pd.DataFrame(metrics_list)
ebm_importance_runs = np.vstack(ebm_importance_runs)  # shape: [N_SEEDS, n_features]
xgb_importance_runs = np.vstack(xgb_importance_runs)

print("\n=== Performance across seeds (first 5 rows) ===")
display(metrics_df.head())

print("\n=== Mean performance across seeds ===")
display(metrics_df.mean(numeric_only=True))

print("\n=== Std dev of performance across seeds ===")
display(metrics_df.std(numeric_only=True))


Running seed 1 ...
Running seed 2 ...
Running seed 3 ...
Running seed 4 ...
Running seed 5 ...
Running seed 6 ...
Running seed 7 ...
Running seed 8 ...
Running seed 9 ...
Running seed 10 ...
Running seed 11 ...
Running seed 12 ...
Running seed 13 ...
Running seed 14 ...
Running seed 15 ...
Running seed 16 ...
Running seed 17 ...
Running seed 18 ...
Running seed 19 ...
Running seed 20 ...

=== Performance across seeds (first 5 rows) ===


,seed,EBM_AUC,EBM_ACC,EBM_F1,EBM_Brier,XGB_AUC,XGB_ACC,XGB_F1,XGB_Brier
0,1,0.799652,0.733071,0.722053,0.181380,0.801070,0.734286,0.724158,0.180645
1,2,0.799612,0.733214,0.722532,0.181414,0.801163,0.734286,0.724404,0.180640
2,3,0.799636,0.732857,0.722057,0.181396,0.801014,0.734000,0.724066,0.180693
3,4,0.799770,0.733214,0.722532,0.181320,0.800961,0.733714,0.723442,0.180676
4,5,0.799635,0.733857,0.722892,0.181333,0.801248,0.733929,0.723603,0.180582



=== Mean performance across seeds ===


,0
seed,10.500000
EBM_AUC,0.799723
EBM_ACC,0.733146
EBM_F1,0.722235
EBM_Brier,0.181354
XGB_AUC,0.801043
XGB_ACC,0.734336
XGB_F1,0.724210
XGB_Brier,0.180663



=== Std dev of performance across seeds ===


,0
seed,5.916080
EBM_AUC,0.000081
EBM_ACC,0.000344
EBM_F1,0.000404
EBM_Brier,0.000029
XGB_AUC,0.000157
XGB_ACC,0.000668
XGB_F1,0.000671
XGB_Brier,0.000061


In [9]:
# ===========================================================
# SECTION 6: GLOBAL EXPLANATION STABILITY (VARIANCE & RANK)
# ===========================================================

def compute_stability_stats(importance_matrix, feature_names):
    """
    importance_matrix: shape [n_seeds, n_features]
    Returns:
      - DataFrame with variance across seeds for each feature
      - mean Spearman rank correlation of importance vectors across all seed pairs
    """
    n_seeds, n_features = importance_matrix.shape

    # Normalize each run's importances to sum to 1
    norm_imps = importance_matrix / (importance_matrix.sum(axis=1, keepdims=True) + 1e-12)

    # Variance per feature across seeds
    var_per_feature = np.var(norm_imps, axis=0)
    feature_var_df = pd.DataFrame({
        "feature": feature_names,
        "variance_across_seeds": var_per_feature
    }).sort_values("variance_across_seeds", ascending=True)

    # Mean Spearman correlation of feature importance rankings across seed pairs
    corrs = []
    for i in range(n_seeds):
        for j in range(i + 1, n_seeds):
            corr, _ = spearmanr(norm_imps[i, :], norm_imps[j, :])
            corrs.append(corr)

    mean_spearman = np.mean(corrs)
    return feature_var_df, mean_spearman

# Compute stability stats
ebm_var_df, ebm_mean_spearman = compute_stability_stats(ebm_importance_runs, feature_names)
xgb_var_df, xgb_mean_spearman = compute_stability_stats(xgb_importance_runs, feature_names)

print("=== Mean Spearman rank correlation (global feature importance) ===")
print(f"EBM               : {ebm_mean_spearman:.4f}")
print(f"XGBoost + SHAP    : {xgb_mean_spearman:.4f}")

print("\n=== EBM - features with LOWEST variance (most stable) ===")
display(ebm_var_df.head(10))

print("\n=== XGBoost+SHAP - features with LOWEST variance (most stable) ===")
display(xgb_var_df.head(10))

print("\n=== EBM - features with HIGHEST variance (least stable) ===")
display(ebm_var_df.tail(10))

print("\n=== XGBoost+SHAP - features with HIGHEST variance (least stable) ===")
display(xgb_var_df.tail(10))


=== Mean Spearman rank correlation (global feature importance) ===
EBM               : 1.0000
XGBoost + SHAP    : 0.9969

=== EBM - features with LOWEST variance (most stable) ===


,feature,variance_across_seeds
9,active,1.439146e-11
0,gender,1.991545e-11
7,smoke,3.264190e-11
8,alco,1.308805e-10
5,cholesterol,2.160728e-10
6,gluc,2.657832e-10
1,height,4.620823e-07
3,ap_hi,1.438624e-06
2,weight,2.146274e-06
4,ap_lo,3.756391e-06



=== XGBoost+SHAP - features with LOWEST variance (most stable) ===


,feature,variance_across_seeds
8,alco,6.583055e-08
7,smoke,1.092101e-07
9,active,1.207692e-07
6,gluc,1.262253e-07
0,gender,3.298869e-07
5,cholesterol,8.279221e-07
1,height,9.729462e-07
10,age_years,1.168913e-06
11,BMI,2.827599e-06
2,weight,3.302994e-06



=== EBM - features with HIGHEST variance (least stable) ===


,feature,variance_across_seeds
7,smoke,3.264190e-11
8,alco,1.308805e-10
5,cholesterol,2.160728e-10
6,gluc,2.657832e-10
1,height,4.620823e-07
3,ap_hi,1.438624e-06
2,weight,2.146274e-06
4,ap_lo,3.756391e-06
10,age_years,8.615941e-06
11,BMI,1.554541e-05



=== XGBoost+SHAP - features with HIGHEST variance (least stable) ===


,feature,variance_across_seeds
9,active,1.207692e-07
6,gluc,1.262253e-07
0,gender,3.298869e-07
5,cholesterol,8.279221e-07
1,height,9.729462e-07
10,age_years,1.168913e-06
11,BMI,2.827599e-06
2,weight,3.302994e-06
4,ap_lo,8.054466e-05
3,ap_hi,9.958568e-05
